# MECANISMOS DE ACCIÓN - Análisis exploratorio de los datos (EDA)

In [ ]:
# pip install opendatasets

**NOMBRES**: 
* Andrea Marcela Castrillon Buitrago 
* Yeison Fernando Villamil Franco

En el pasado, cuando se trabajaba en la creación de un medicamento este solo era probado en personas pero no se realizaba una investigación exhaustiva del comportamiento genético y celular del medicamento. Básicamente, un medicamente intenta modular la proteína que es activada cuando se presenta una enfermedad. El mecanismo de acción o MoA por sus siglas en inglés muestra las diferentes activaciones de proteínas al tratamiento con fármacos. Las variables de entrada son las respuestas (expresiones génicas y viabilidad celular) al tratamiento y las variables de salida son un conjuntos de proteínas que se ven activadas son el tratamiento. Valores binarios: 0 y 1 muestran la no activación y activación.

Los datos se encuentran en un concurso de kaggle, usando el siguiente link:

* https://www.kaggle.com/c/lish-moa/data

Fue usado como fuente de programación un notebook en jupyter y los datos fueron cargados de manera directa descargando los archivos en la máquina persona y cargándolos usando la misma carpeta para notebook y archivos de datos.

## Carga de datos

In [ ]:
#Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import scipy.stats as stats
import opendatasets as od

In [ ]:
train_features = pd.read_csv('train_features.csv')
train_target = pd.read_csv('train_targets_scored.csv')
test_features = pd.read_csv('test_features.csv')

*El dataset de entrenamiento y prueba consiste en filas que representan las muestras (fármacos usados), `cp_type`: es el tipo de perturbación al comportamiento celular --> trt_cp: perturbación química (uso de fármacos) y ctl_vehicle: perturbación de control. `cp_time`: es el tiempo en horas del suministro del fármaco y `cp_dose`: D1 es dosis baja y D2 es dosis alta (no es indicada la cantidad en volumen suministrado, solo se muestra como una dosis baja y alta). Las variables de entrada con respuestas representadas en expresiones génicas y viabilidad celular, siendo el comportamiento medido cuando se usan fármacos para moderar las proteínas.*

In [ ]:
train_features.head(5)

In [ ]:
test_features.head(5)

In [ ]:
train_target.head(5)

## Análisis - dimensiones

In [ ]:
#Información de cada uno de los archivos
print(train_features.info())
print("---------------------------------------")
print(test_features.info())
print("---------------------------------------")
print(train_target.info())

In [ ]:
print('Las dimensiones de los datos de entranamiento son:',train_features.shape)
print("---------------------------------------------------------------")
print('Las dimensiones de los labels de entranamiento son:', train_target.shape)
print("----------------------------------------------------------------")
print('Las dimensiones de los datos de prueba son:',test_features.shape)

*Las expresiones génicas en variables de entrada son 772, viabilidad celular son 100. Para el caso de las variables de salidas se presentan 206 proteínas diferentes que fueron activadas o no para las 23814 muestras o fármacos.*

In [ ]:
## Existen valores nulos?
print('Valores nulos de los datos de entrenamiento:',train_features.isnull().sum().sum())
print('--')
print('Valores nulos de los target de entrenamiento:',train_target.isnull().sum().sum())
print('--')
print('Valores nulos de los datos de prueba:',test_features.isnull().sum().sum())

In [ ]:
train_features.describe()

# 1. Resumen de los datos

## Datos de entrenamiento

In [ ]:
### Resumen de los datos de entrenamiento
categorical = ['cp_type', 'cp_dose']
num_plot = len(categorical)
total_cols = 3
total_rows = 1 

fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(16,6))
sns.countplot('cp_type',data=train_features,ax=axs[0])
axs[0].set_title('cp_type - Datos de entrenamiento')
sns.countplot('cp_dose',data=train_features,ax=axs[1])
axs[1].set_title('cp_dose - Datos de entrenamiento')
sns.countplot('cp_time',data=train_features,ax=axs[2])
axs[2].set_title('cp_time - Datos de entrenamiento')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16,12))
sns.countplot('cp_type',hue='cp_dose',data=train_features,ax=axs[0,0],palette='husl')
axs[0,0].set_title('Dosis y tipo de perturbación - Datos de entrenamiento')
sns.countplot('cp_type',hue='cp_time',data=train_features,ax=axs[0,1],palette='husl')
axs[0,1].set_title('Tiempo de dosis y tipo de perturbación - Datos de entrenamiento')
sns.countplot('cp_dose',hue='cp_time',data=train_features,ax=axs[1,0],palette='husl')
axs[1,0].set_title('Tiempo y carga de la dosis')
sns.countplot('cp_dose',hue='cp_type',data=train_features,ax=axs[1,1],palette='husl')
axs[1,1].set_title('Tiempo y tipos de dosis')

### Distribuciones - Expresiones génicas

*Debido a que hay 772 expresiones génicas, son tomadas 12 aleatoriamente para conocer sus distribuciones. Adicionalmente, son mostrados dos histogramas con las medias y desviaciones estandar de todas las expresiones génicas.*

In [ ]:
import matplotlib
import random
from scipy.stats import norm, t

matplotlib.rcParams.update({'font.size': 14, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
num = 12
genes = [col for col in train_features.columns if col.startswith('g-')]
sampleddata = random.sample(list(genes),num)
num_plot = len(sampleddata)
total_cols = 2
total_rows = num_plot//total_cols 
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(6*total_cols, 5*total_rows), constrained_layout=True)

for i, var in enumerate(sampleddata):
    rows = i//total_cols
    pos = i% total_cols
    
    plot = sns.distplot(train_features[sampleddata], kde=True, fit=t, ax= axs[rows][pos], fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 1, "linewidth": 2}, hist_kws={"linewidth": 1,"alpha": 1, "color": "g", "label": "Hist"},kde_kws={"color": "red", "linewidth": 1.5, "label": "kde"})
    plot.set_title(var)
    plot.legend()

In [ ]:
# from scipy.stats import norm, t
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(train_features[genes].mean(), kde=True, fit=t, fit_kws={'color': 'r', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'blue'}, kde_kws = {'label':'kde', 'color':'k'})
plt.title('Distribución de las medias - Expresiones génicas')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(train_features[genes].std(), kde=True, fit=t, fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.2, 'label': 'Hist', 'color': 'r'}, kde_kws={'label':'kde', 'color': 'purple'})
plt.title('Distribución de la desviación estandar - Expresiones génicas')
plt.legend()
plt.show()

### Distribuciones - viabilidad celular

In [ ]:
num = 12
cells = [col for col in train_features.columns if col.startswith('c-')]
sampleddata_c = random.sample(list(cells),num)
num_plot = len(sampleddata_c)
total_cols = 2
total_rows = num_plot//total_cols 
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(6*total_cols, 5*total_rows), constrained_layout=True)

for i, var in enumerate(sampleddata_c):
    rows = i//total_cols
    pos = i% total_cols
    
    plot = sns.distplot(train_features[sampleddata_c], kde=True, fit=t, ax= axs[rows][pos], fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 1, "linewidth": 2}, hist_kws={"linewidth": 1,"alpha": 1, "color": "r", "label": "Hist"},kde_kws={"color": "blue", "linewidth": 1.5, "label": "kde"})
    plot.set_title(var)
    plot.legend()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(train_features[cells].mean(), kde=True, fit=t, fit_kws={'color': 'r', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'blue'}, kde_kws = {'label':'kde', 'color': 'blue'})
plt.title('Distribución de las medias - Viabilidad celular')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(train_features[cells].std(), kde=True, fit=t, fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'r'}, kde_kws={'label': 'kde', 'color': 'blue'})
plt.title('Distribución de la desviación estandar - Viabilidad celular')
plt.legend()
plt.show()

## Datos de prueba

In [ ]:
### Resumen de datos de prueba
### Resumen de los datos de entrenamiento
categorical = ['cp_type', 'cp_dose']
num_plot = len(categorical)
total_cols = 3
total_rows = 1 

fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(16,6))
sns.countplot('cp_type',data=test_features,ax=axs[0])
axs[0].set_title('cp_type - Datos de prueba')
sns.countplot('cp_dose',data=test_features,ax=axs[1])
axs[1].set_title('cp_dose - Datos de prueba')
sns.countplot('cp_time',data=test_features,ax=axs[2])
axs[2].set_title('cp_time - Datos de prueba')

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16,12))
sns.countplot('cp_type',hue='cp_dose',data=test_features,ax=axs[0,0],palette='husl')
axs[0,0].set_title('Dosis y tipo de perturbación - Datos de entrenamiento')
sns.countplot('cp_type',hue='cp_time',data=test_features,ax=axs[0,1],palette='husl')
axs[0,1].set_title('Tiempo de dosis y tipo de perturbación - Datos de entrenamiento')
sns.countplot('cp_dose',hue='cp_time',data=test_features,ax=axs[1,0],palette='husl')
axs[1,0].set_title('Tiempo y carga de la dosis')
sns.countplot('cp_dose',hue='cp_type',data=test_features,ax=axs[1,1],palette='husl')
axs[1,1].set_title('Tiempo y tipos de dosis')

### Distribuciones - expresiones génicas

In [ ]:
num_plot = len(sampleddata)
total_cols = 2
total_rows = num_plot//total_cols 
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(6*total_cols, 5*total_rows), constrained_layout=True)

for i, var in enumerate(sampleddata):
    rows = i//total_cols
    pos = i% total_cols
    
    plot = sns.distplot(test_features[sampleddata], kde=True, fit=t, ax= axs[rows][pos], fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 1, "linewidth": 2}, hist_kws={"linewidth": 1,"alpha": 1, "color": "g", "label": "Hist"},kde_kws={"color": "red", "linewidth": 1.5, "label": "kde"})
    plot.set_title(var)
    plot.legend()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(test_features[genes].mean(), kde=True, fit=t, fit_kws={'color': 'r', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'blue'}, kde_kws ={'label':'kde', 'color': 'blue'})
plt.title('Distribución de las medias - Expresiones génicas')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(test_features[genes].std(), kde=True, fit=t, fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'r'}, kde_kws = {'label':'kde', 'color':'blue'})
plt.title('Distribución de la desviación estandar - Expresiones génicas')
plt.legend()
plt.show()

### Distribuciones - viabilidad celular

In [ ]:
num_plot = len(sampleddata_c)
total_cols = 2
total_rows = num_plot//total_cols 
fig, axs = plt.subplots(nrows=total_rows, ncols=total_cols, figsize=(6*total_cols, 5*total_rows), constrained_layout=True)

for i, var in enumerate(sampleddata_c):
    rows = i//total_cols
    pos = i% total_cols
    
    plot = sns.distplot(test_features[sampleddata_c], kde=True, fit=t, ax= axs[rows][pos], fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 1, "linewidth": 2}, hist_kws={"linewidth": 1,"alpha": 1, "color": "r", "label": "Hist"},kde_kws={"color": "blue", "linewidth": 1.5, "label": "kde"})
    plot.set_title(var)
    plot.legend()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(test_features[cells].mean(), kde=True, fit=t, fit_kws={'color': 'r', 'label': 'norm', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'blue'}, kde_kws = {'label': 'kde', 'color': 'blue'})
plt.title('Distribución des la medias - Viabilidad celular')
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(10,6))
sns.distplot(test_features[cells].std(), kde=True, fit=t, fit_kws={'color': 'k', 'label': 't.std', 'alpha' : 0.9}, hist_kws = {'alpha': 0.25, 'label': 'Hist', 'color': 'r'}, kde_kws = {'label': 'kde', 'color': 'blue'})
plt.title('Distribución de la desviación estandar - viabilidad celular')
plt.legend()
plt.show()

## Análisis - Proteínas (variables de salida)

In [ ]:
## Target de prueba
X = train_features.iloc[:,5:104]
y = train_target.iloc[:,1:207]
# train_features.head(5)
# train_target.head(5)
# El objetivo es ver cuantas veces se repite el valor '1' en cada uno de los fármacos
sumatoria = y.sum()
datos_y = pd.DataFrame(data= sumatoria, columns = ['Conteo'])
datos_y = datos_y.sort_values(by = 'Conteo', ascending = False)
# datos_y.head()

In [ ]:
fig = plt.figure(figsize=(12, 60))

sns.barplot(x=y.sum(axis=0).sort_values(ascending=False).values,
            y=y.sum(axis=0).sort_values(ascending=False).index)

plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.xlabel('')
plt.ylabel('')
plt.title('Conteo de proteínas para el archivo de entrenamiento', size=18, pad=18)

plt.show()

**Los tipos de MoA son los siguientes:**
* Receptores: Targets que se encuentran en la superficie y que pueden ser modulados positivamente por `agonistas` y negativamente por `antagonistas`
* Enzimas: Se encuentran dentro de la célula. Son moduladas positivamente por `activadores` y negativamente por `inhibidores` y `bloqueadores`
* Agentes: Son sencillamente moduladores

In [ ]:
###Tipos de fármacos usados
train_target.head(5)

### Agentes moduladores

In [ ]:
### Agentes moduladores
target=train_target.iloc[:,1:].sum(axis=0)
fig = plt.figure(figsize=(12, 6))
agentes = target.loc[target.index.str.contains('_agent')]
agentes_data = pd.DataFrame(agentes)
sns.barplot(agentes.values, agentes.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Agentes moduladores', size=18, pad=18)
plt.show()

### Receptores

**Receptores positivios**

In [ ]:
# Receptores positivos
fig = plt.figure(figsize=(12, 12))
receptor = target.loc[target.index.str.contains('_agonist')].sort_values(ascending=False)
sns.barplot(receptor.values, receptor.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Receptores Positivos', size=18, pad=18)
plt.show()

**Receptores negativos**

In [ ]:
# Receptores negativos
receptor_ant = target.loc[target.index.str.contains('_antagonist')].sort_values(ascending=False)
fig = plt.figure(figsize=(12, 12))
sns.barplot(receptor_ant.values, receptor_ant.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Receptores Negativos', size=18, pad=18)
plt.show()

### Enzimas

**Enzimas positivas**

In [ ]:
##Enzimas positivas
enzimas_pos = target.loc[target.index.str.contains('_activator')].sort_values(ascending=False)
fig = plt.figure(figsize=(12, 6))
sns.barplot(enzimas_pos.values, enzimas_pos.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Enzimas positivas', size=18, pad=18)
plt.show()

**Enzimas negativas**

In [ ]:
##Enzimas negativas
enzimas_neg = target.loc[target.index.str.contains('_inhibitor')].sort_values(ascending=False)
fig = plt.figure(figsize=(12, 30))
sns.barplot(enzimas_neg.values, enzimas_neg.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Enzimas negativas (Inhibidores)', size=18, pad=18)
plt.show()

In [ ]:
enzimas_neg_2 = target.loc[target.index.str.contains('_blocker')].sort_values(ascending=False)
fig = plt.figure(figsize=(12, 3))
sns.barplot(enzimas_neg_2.values, enzimas_neg_2.index)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)
plt.title('Enzimas negativas (Bloqueadoras)', size=18, pad=18)
plt.show()

# 2. Estadísticos (test)

## Matriz de correlación

### Datos de entrenamiento 

In [ ]:
#Matriz de correlación 
g_features = [col for col in train_features.columns if col.startswith('g-')]
datos_g = train_features.iloc[:,4:876]
matrix_g = pd.DataFrame(data = datos_g, columns = [col for col in train_features.columns if col.startswith('g-')])

In [ ]:
# datos_g1 = train_features.iloc[:,4:44]
Matriz_Correlacion = matrix_g.corr()
fig = plt.figure(figsize=(20,20))
ax = sns.heatmap(Matriz_Correlacion,annot=False,square=True)#, xticklabels = features, yticklabels = features)
ax.tick_params(axis='x', labelsize=20, rotation=90, pad=20)
ax.tick_params(axis='y', labelsize=20, rotation=0, pad=20)

for idx, label in enumerate(ax.get_xticklabels()):
    if idx % 5 == 0:
        label.set_visible(True)
    else:
        label.set_visible(False)
        
for idx, label in enumerate(ax.get_yticklabels()):
    if idx % 5 == 0:
        label.set_visible(True)
    else:
        label.set_visible(False)
        
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20, pad=10)

plt.title('Correlación de expresiones génicas', size=25, pad=25)
plt.show()
# Matriz_Correlacion

In [ ]:
#Correlación para la viabilidad de celulas
c_features = [col for col in train_features.columns if col.startswith('c-')]
datos_c = train_features.iloc[:,4:876]
matrix_c = pd.DataFrame(data = datos_g, columns = [col for col in train_features.columns if col.startswith('c-')])

In [ ]:
Matriz_Correlacion_C = matrix_c.corr()
fig = plt.figure(figsize=(20,20))
ax = sns.heatmap(Matriz_Correlacion_C,annot=False,square=True)#, xticklabels = features, yticklabels = features)
ax.tick_params(axis='x', labelsize=20, rotation=90, pad=20)
ax.tick_params(axis='y', labelsize=20, rotation=0, pad=20)

for idx, label in enumerate(ax.get_xticklabels()):
    if idx % 5 == 0:
        label.set_visible(True)
    else:
        label.set_visible(False)
        
for idx, label in enumerate(ax.get_yticklabels()):
    if idx % 5 == 0:
        label.set_visible(True)
    else:
        label.set_visible(False)
        
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20, pad=10)

plt.title('Correlación de viabilidad celular', size=25, pad=25)
plt.show()